In [3]:
import time
import glob
import pandas as pd
import os

In [4]:
os.chdir('/home/wscho/research_etc_proj/gk2aICT/')

from src.config import (
    LOCAL_GK2A_DIR,
    LATLON_DIR
)
from src.variables import (
    VAR2DSKEY,
    FD_VAR2FILE,
)
from src.tools.clip import (
    GK2ABaseProcessor,
    GK2AFDProcessor,
    GK2AEAProcessor,
    GK2AKOProcessor,

)
from src.utils import (
    read_gk2a_data
)

In [5]:
##config data 
# 준비 데이터1 : REF_TIME 및 그 전 2개 time에 해당하는 gk2a CLA 데이터(총 3개의 위성 데이터)
# 준비 데이터2 : ASOS 및 발전소 GHI 및 각각의 Metadata

REF_TIME         = '2022-05-03 12:50'        # 이용할 위성 시간대 timezone = Seoul
INTERVAL         =  15                       # 예보 시간 interval(분)
FORECAST_START   =  120                      # REF_TIME 기준 예보 시작 시간(분)
FORECAST_END     =  240                      # REF_TIME 기준 예보 종료 시간(분)

# 구름 이동 산출물 저장 장소
GK2A_RAW_PATH    = '/mnt/sdb1/wscho/Jobs/gk2aICT/gk2a_raw.pkl'
GK2A_CLIP_PATH    = '/mnt/sdb1/wscho/Jobs/gk2aICT/gk2a_clip.pkl'

# 위성데이터 존재 directory
BASE_DIR         = '/mnt/sdb1/wscho/data_for_research/ICTgk2a/'

#Boundary 설정 옵션
BOUNDARY_OPTION  = False
BOUNDARY_SIZE = 250
BOUNDARY_SIZE *= BOUNDARY_OPTION #Do not change this code for checking Option & size correction

#target 지역
KO_RANGE = {
    'ullatitude': 42.0,
    'ullongitude': 124.0,
    'lrlatitude': 33.0,
    'lrlongitude': 130.0,
}

In [6]:
#class 선언 시에 위성 resolution 지정 필요
s = time.time()
ea_gk2a = GK2AEAProcessor(resolution = 2.0)

##### PREDICT CLOUD AMOUNT ####

In [7]:
# get gk2a variable
var_set = ea_gk2a.get_gk2a_var_set('CLA', 'CA',GK2A_RAW_PATH,BASE_DIR,REF_TIME)

using data named 
'/mnt/sdb1/wscho/data_for_research/ICTgk2a/20220503/LE2/cla/gk2a_ami_le2_cla_ea020lc_202205030330.nc' 
for time '2022-05-03 12:30:00'
using data named 
'/mnt/sdb1/wscho/data_for_research/ICTgk2a/20220503/LE2/cla/gk2a_ami_le2_cla_ea020lc_202205030340.nc' 
for time '2022-05-03 12:40:00'
using data named 
'/mnt/sdb1/wscho/data_for_research/ICTgk2a/20220503/LE2/cla/gk2a_ami_le2_cla_ea020lc_202205030350.nc' 
for time '2022-05-03 12:50:00'


In [8]:
# clip to Korean Peninsula
var_clip = [ea_gk2a.cut_with_latlon(cld_x, **KO_RANGE, boundary = BOUNDARY_OPTION) for cld_x in var_set]

In [9]:
var_dic = {
            'DOMAIN' : KO_RANGE,
            'LATLON' : [ea_gk2a.lat_clip, ea_gk2a.lon_clip],
            'BOUNDARY_SIZE' : BOUNDARY_SIZE,
            'DATA' : var_clip,
}

In [10]:
pd.Series(var_dic).to_pickle(GK2A_CLIP_PATH)

In [11]:
e = time.time()
print('='*22*3)
print(f'Time for loading and clipping gk2a Images : {(e-s)} sec')
print('='*22*3)

Time for loading and clipping gk2a Images : 3.2318971157073975 sec
